In [ ]:
from langchain_community.document_loaders import TextLoader
#from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter

#from langchain.embeddings import openai
#from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

loader = TextLoader("lease-11-1958.txt", encoding="utf-8")
docs = loader.load()
#print(len(docs))
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=80)
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

chunks = text_splitter.split_documents(docs)

load_dotenv()

#embeddings = openai.OpenAIEmbeddings()
#vector = embeddings.embed_query("Hi! My name is Mr.Embedding")

'''
from transformers import pipeline
checkpoint = "facebook/bart-base"
feature_extractor = pipeline("feature-extraction",framework="pt",model=checkpoint)
text = "Transformers is an awesome library!"

#Reducing along the first dimension to get a 768 dimensional array
feature_extractor(text,return_tensors = "pt")[0].numpy().mean(axis=0)
'''

#len(chunks)
#chunks[0].page_content
texts = [c.page_content for c in chunks]
print(texts)

In [ ]:
import os

def create_db_connection():
    params = db_config()
    try:
        conn = psycopg2.connect(**params)
        return conn
    except (Exception, psycopg2.Error) as error:
        print("Error while connecting", error)
    return None


import requests

model_id = "sentence-transformers/all-MiniLM-L6-v2"
api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
hf_token = os.getenv('HF_ACCESS_TOKEN')
headers = {"Authorization": f"Bearer {hf_token}"}

def query(texts):
    response = requests.post(api_url, headers=headers, json={"inputs": texts, "options":{"wait_for_model":True}})
    return response.json()


output = query(texts)

import pandas as pd
embeddings = pd.DataFrame(output)
print(embeddings)